In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.layers import Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Input, ZeroPadding2D, Activation
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Add, MaxPooling2D, Flatten, Dense, Softmax, Activation, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD

import numpy as np
import h5py

In [2]:
tf.__version__
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
def dataGenerator(n_samples=0, start_index=-1, stop_index=-1,shuffle=True):
    def generator():
        if (start_index>0 and stop_index>0):
            indexes = np.arange(start_index, stop_index)
        else:
            indexes = np.arange(n_samples)
        if shuffle:
            np.random.shuffle(indexes)
        
        for i in indexes:
            x = np.array(hdf5['xs'][i])
            y = np.array(hdf5['ys'][i])
            x = np.moveaxis(x,0,-1)
            yield x, y
    return generator

In [4]:
hdf5 = h5py.File('./hdf5/kyu_dataset_detlef.hdf5','r')
# for j in range (0, 13):
#     print(hdf5['xs'][50][j])
batch_size = 64
dataset_size = hdf5['xs'].shape[0]
val_start = 0.7
val_stop = 0.8
train_steps = int(dataset_size * (1 - val_start)/batch_size)
val_steps = int(dataset_size * (val_stop - val_start)/batch_size)

data_gen = dataGenerator(n_samples=int(dataset_size * val_start), shuffle=True) # 90% of the complete dataset
dataset = tf.data.Dataset.from_generator(data_gen, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,13)),tf.TensorShape((1))))
dataset = dataset.batch(batch_size, drop_remainder=True)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

data_gen_valid = dataGenerator(start_index=int(dataset_size * val_start), # 10% of the complete dataset
                               stop_index=int(dataset_size * val_stop), 
                               shuffle=False)
dataset_valid = tf.data.Dataset.from_generator(data_gen_valid, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,13)),tf.TensorShape((1))))
dataset_valid = dataset_valid.batch(batch_size, drop_remainder=True)
dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)


In [9]:
int(hdf5['xs'].shape[0]/100)*1

135531

In [10]:
def residual_block(x, filters, kernel_size):
    bn_axis = 3
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(x)
    y = BatchNormalization(axis=bn_axis)(y)
    y = ReLU()(y)
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(y)
    y = BatchNormalization(axis=bn_axis)(y)
    output = Add()([x,y])
    output = ReLU()(output)
    return output

def go_res():
    bn_axis = 3
    inputs = Input(shape=(19, 19, 13))
    conv5x5 = Conv2D(kernel_size=5,
                     filters=256,
                     padding="same",
                     name='conv5x5')(inputs)
    conv5x5 = BatchNormalization(axis=bn_axis)(conv5x5)
    conv1x1 = Conv2D(kernel_size=1,
                     filters=256,
                     padding="same",
                     name='conv1x1')(inputs)
    conv1x1 = BatchNormalization(axis=bn_axis)(conv1x1)
    outputs = Add()([conv5x5, conv1x1])
    outputs = ReLU()(outputs)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = residual_block(x=outputs,
                             filters=256,
                             kernel_size=3)
    outputs = Conv2D(kernel_size=3,
                     filters=1,
                     padding="same")(outputs)
    outputs = BatchNormalization(axis=bn_axis)(outputs)
    outputs = ReLU()(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


model = go_res()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 19, 19, 13)  0           []                               
                                ]                                                                 
                                                                                                  
 conv5x5 (Conv2D)               (None, 19, 19, 256)  83456       ['input_3[0][0]']                
                                                                                                  
 conv1x1 (Conv2D)               (None, 19, 19, 256)  3584        ['input_3[0][0]']                
                                                                                                  
 batch_normalization_16 (BatchN  (None, 19, 19, 256)  1024       ['conv5x5[0][0]']          

                                                                                                  
 add_28 (Add)                   (None, 19, 19, 256)  0           ['re_lu_22[0][0]',               
                                                                  'batch_normalization_25[0][0]'] 
                                                                                                  
 re_lu_24 (ReLU)                (None, 19, 19, 256)  0           ['add_28[0][0]']                 
                                                                                                  
 conv2d_22 (Conv2D)             (None, 19, 19, 256)  590080      ['re_lu_24[0][0]']               
                                                                                                  
 batch_normalization_26 (BatchN  (None, 19, 19, 256)  1024       ['conv2d_22[0][0]']              
 ormalization)                                                                                    
          

In [5]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # 這邊就是圖5上的1x1x64降維操作，假設input x的維度是(n, n, 256), channel last
    x =  Conv2D(filters1, (1, 1),
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x =  BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x =  Activation('relu')(x)
    
    # 正常的3x3x64卷積操作，Feature Map長寬仍是n x n
    x =  Conv2D(filters2, kernel_size,
                      padding='same',
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x =  BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x =  Activation('relu')(x)
    
    # 最後升維到256，維度(n,n,256) -> 變成可以和(Indentity)input x相加的維度
    x =  Conv2D(filters3, (1, 1),
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x =  BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
    
    # 相加後做non-linear轉換
    x =  Add()([x, input_tensor])
    x =  Activation('relu')(x)
    return x
def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(1, 1)):
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 因為是projection shortcut 所以input的x可能跟output維度不同
    # input維度(n,n,256) -->降維 (n,n,64)
    # 如果Strides有改，則利用Strides來改變Feature Map長寬
    x =  Conv2D(filters1, (1, 1), strides=strides,  
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x =  BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x =  Activation('relu')(x)
    
    # (3,3)的kernel, padding都pad好pad滿，不改變Feature Map尺寸大小
    x =  Conv2D(filters2, kernel_size, padding='same', 
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x =  Activation('relu')(x)
    
    # 用1x1 conv升維到假設512
    x = Conv2D(filters3, (1, 1), 
#                       kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
    
    # 因input維度是256，這邊就需要做projectr將維度升到512相加
    shortcut = Conv2D(filters3, (1, 1), strides=strides,
#                              kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)
    # F(x) + x(升維後的x)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x
def ResNet50(include_top=True,
             input_tensor=None,
             input_shape=None,
             pooling=False):
    img_input = Input(shape = input_shape)
    bn_axis = 3 #unknow
    ##### optional
#     x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
#     x = layers.Conv2D(32, (7, 7),
#                       strides=(2, 2),
#                       padding='same',
#                       kernel_initializer='he_normal',
#                       name='conv1')(img_input)
#     x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
#     x = layers.Activation('relu')(x)
    #####
    conv5x5 = Conv2D(kernel_size=5,
                     filters=256,
                     padding="same",
                     name='conv5x5')(img_input)
    conv1x1 = Conv2D(kernel_size=1,
                     filters=256,
                     padding="same",
                     name='conv1x1')(img_input)
    outputs = Add()([conv5x5, conv1x1])
    outputs = ReLU()(outputs)
    x = conv_block(outputs, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))  # input Channel大小會跟最後最後residual output尺寸一樣
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    # 256-d to 512-d
    x = conv_block(x, 3, [256, 256, 1024], stage=3, block='a') # projection shortcut
    x = identity_block(x, 3, [256, 256, 1024], stage=3, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=3, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=3, block='d')
    # 512-d to 1024-d
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a') # projection shortcut
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')
    # 1024-d to 2048-d
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    
#     x = GlobalAveragePooling2D(name='avg_pool')(x)
#     x = Dense(361, activation='softmax', name='fc1000')(x)
    outputs = Conv2D(kernel_size=3,
                     filters=1,
                     padding="same")(x)
    outputs = BatchNormalization(axis=bn_axis)(outputs)
    outputs = ReLU()(outputs)
    outputs = Flatten()(outputs)
    x = Softmax()(outputs)
#     if include_top:
#         x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
#         x = layers.Dense(classes, activation='softmax', name='fc1000')(x)
#     else:
#         if pooling == 'avg':
#             x = layers.GlobalAveragePooling2D()(x)
#         elif pooling == 'max':
#             x = layers.GlobalMaxPooling2D()(x)
#         else:
#             warnings.warn('The output shape of `ResNet50(include_top=False)` '
#                           'has been changed since Keras 2.2.0.')
#     x = layers.GlobalAveragePooling2D()(x)
    
    if input_tensor is not None:
        inputs = keras_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input
        
    model = Model(inputs, x, name='resnet50')
    return model
model = ResNet50(include_top=True,
                 input_tensor=None,
                 input_shape=(19, 19, 13),
                 pooling=False)
opt = Nadam(learning_rate = 0.0001)
model.compile(optimizer = opt,
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 19, 19, 13)  0           []                               
                                ]                                                                 
                                                                                                  
 conv5x5 (Conv2D)               (None, 19, 19, 256)  83456       ['input_1[0][0]']                
                                                                                                  
 conv1x1 (Conv2D)               (None, 19, 19, 256)  3584        ['input_1[0][0]']                
                                                                                                  
 add (Add)                      (None, 19, 19, 256)  0           ['conv5x5[0][0]',         

                                                                                                  
 bn2c_branch2c (BatchNormalizat  (None, 19, 19, 256)  1024       ['res2c_branch2c[0][0]']         
 ion)                                                                                             
                                                                                                  
 add_3 (Add)                    (None, 19, 19, 256)  0           ['bn2c_branch2c[0][0]',          
                                                                  'activation_5[0][0]']           
                                                                                                  
 activation_8 (Activation)      (None, 19, 19, 256)  0           ['add_3[0][0]']                  
                                                                                                  
 res3a_branch2a (Conv2D)        (None, 19, 19, 256)  65792       ['activation_8[0][0]']           
          

                                                                                                  
 bn3c_branch2c (BatchNormalizat  (None, 19, 19, 1024  4096       ['res3c_branch2c[0][0]']         
 ion)                           )                                                                 
                                                                                                  
 add_6 (Add)                    (None, 19, 19, 1024  0           ['bn3c_branch2c[0][0]',          
                                )                                 'activation_14[0][0]']          
                                                                                                  
 activation_17 (Activation)     (None, 19, 19, 1024  0           ['add_6[0][0]']                  
                                )                                                                 
                                                                                                  
 res3d_bra

                                )                                                                 
                                                                                                  
 bn4b_branch2c (BatchNormalizat  (None, 19, 19, 1024  4096       ['res4b_branch2c[0][0]']         
 ion)                           )                                                                 
                                                                                                  
 add_9 (Add)                    (None, 19, 19, 1024  0           ['bn4b_branch2c[0][0]',          
                                )                                 'activation_23[0][0]']          
                                                                                                  
 activation_26 (Activation)     (None, 19, 19, 1024  0           ['add_9[0][0]']                  
                                )                                                                 
          

 add_12 (Add)                   (None, 19, 19, 1024  0           ['bn4e_branch2c[0][0]',          
                                )                                 'activation_32[0][0]']          
                                                                                                  
 activation_35 (Activation)     (None, 19, 19, 1024  0           ['add_12[0][0]']                 
                                )                                                                 
                                                                                                  
 res4f_branch2a (Conv2D)        (None, 19, 19, 256)  262400      ['activation_35[0][0]']          
                                                                                                  
 bn4f_branch2a (BatchNormalizat  (None, 19, 19, 256)  1024       ['res4f_branch2a[0][0]']         
 ion)                                                                                             
          

                                                                                                  
 add_15 (Add)                   (None, 19, 19, 2048  0           ['bn5b_branch2c[0][0]',          
                                )                                 'activation_41[0][0]']          
                                                                                                  
 activation_44 (Activation)     (None, 19, 19, 2048  0           ['add_15[0][0]']                 
                                )                                                                 
                                                                                                  
 res5c_branch2a (Conv2D)        (None, 19, 19, 512)  1049088     ['activation_44[0][0]']          
                                                                                                  
 bn5c_branch2a (BatchNormalizat  (None, 19, 19, 512)  2048       ['res5c_branch2a[0][0]']         
 ion)     

In [6]:
history = model.fit(
    dataset,
    epochs = 1,
    steps_per_epoch = train_steps
#     validation_data = dataset_valid
)

   585/105884 [..............................] - ETA: 10:09:29 - loss: 4.3884 - accuracy: 0.1224

KeyboardInterrupt: 

In [17]:
# model.save('./models/model_kyu_resnet50_withpp.h5')

In [17]:
result = model.evaluate(
    dataset_valid,
    steps = val_steps
)

1309/2117 [=================>............] - ETA: 3:43 - loss: 2.2774 - accuracy: 0.4506

KeyboardInterrupt: 

In [ ]:
model.save('./models/model_kyu_resnet50_withpp_1.h5')

In [27]:
history = model.fit(
    dataset,
    epochs = 1,
    validation_data = val_dataset
)

95270/95270 [==============================] - 14285s 150ms/step - loss: 2.0121 - accuracy: 0.5265


In [28]:
model.save('./models/model_kyu_resnet_2.h5')

In [33]:
history = model.fit(
    dataset,
    epochs = 1,
    validation_data = val_dataset
)

42/42 [==============================] - 8s 188ms/step - loss: 1.8230 - accuracy: 0.5787 - val_loss: 1.9512 - val_accuracy: 0.5478


In [ ]:
model.save('./models/model_kyu_resnet_3.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_resnet_4.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_5.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_6.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_7.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_8.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_9.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_14_11.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!